In [1]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity      
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# import spacy
lemmatizer = nltk.stem.WordNetLemmatizer()

# Download required NLTK data
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [2]:
data = pd.read_csv('Mental_Health_FAQ.csv')
data.drop('Question_ID', axis = 1, inplace = True)
data

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...
93,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [3]:
# Define a function for text preprocessing (including lemmatization)
def preprocess_text(text):
    global tokens
    # Identifies all sentences in the data
    sentences = nltk.sent_tokenize(text)
    
    # Tokenize and lemmatize each word in each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        tokens = [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence) if word.isalnum()]
        # Turns to basic root - each word in the tokenized word found in the tokenized sentence - if they are all alphanumeric 
        # The code above does the following:
        # Identifies every word in the sentence 
        # Turns it to a lower case 
        # Lemmatizes it if the word is alphanumeric

        preprocessed_sentence = ' '.join(tokens)
        preprocessed_sentences.append(preprocessed_sentence)
    
    return ' '.join(preprocessed_sentences)


data['tokenized Questions'] = data['Questions'].apply(preprocess_text)
data.head()

,Questions,Answers,tokenized Questions
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,what doe it mean to have a mental illness
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...,who doe mental illness affect
2,What causes mental illness?,It is estimated that mental illness affects 1 ...,what cause mental illness
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what are some of the warning sign of mental il...
4,Can people with mental illness recover?,"When healing from mental illness, early identi...",can people with mental illness recover


In [4]:
corpus = data['tokenized Questions'].to_list()
corpus

['what doe it mean to have a mental illness',
 'who doe mental illness affect',
 'what cause mental illness',
 'what are some of the warning sign of mental illness',
 'can people with mental illness recover',
 'what should i do if i know someone who appears to have the symptom of a mental disorder',
 'how can i find a mental health professional for myself or my child',
 'what treatment option are available',
 'if i become involved in treatment what do i need to know',
 'what is the difference between mental health professional',
 'how can i find a mental health professional right for my child or myself',
 'if i become involved in treatment what do i need to know',
 'where else can i get help',
 'what should i know before starting a new medication',
 'if i feel better after taking medication doe this mean i am cured and can stop taking it',
 'how can i get help paying for my medication',
 'where can i go to find therapy',
 'where can i learn about type of mental health treatment',
 'wha

In [5]:
# vectorisation
tfidf_vector = TfidfVectorizer()
v_corpus = tfidf_vector.fit_transform(corpus)
print(v_corpus)

  (0, 101)	0.36441743462193266
  (0, 128)	0.2595599997967565
  (0, 93)	0.4131356340148716
  (0, 216)	0.3064104920291124
  (0, 126)	0.4362655661041626
  (0, 110)	0.3782586235113424
  (0, 70)	0.4131356340148716
  (0, 228)	0.1765554179540623
  (1, 6)	0.5523407361335997
  (1, 231)	0.5066540480622876
  (1, 101)	0.39613691788888045
  (1, 128)	0.2821525222397721
  (1, 70)	0.44909562820041293
  (2, 36)	0.6958986765070116
  (2, 101)	0.5441013605412818
  (2, 128)	0.3875416915165623
  (2, 228)	0.2636098989594234
  (3, 189)	0.3888505175982014
  (3, 226)	0.3888505175982014
  (3, 208)	0.20619723379229024
  (3, 142)	0.5390782138680418
  (3, 192)	0.3888505175982014
  (3, 20)	0.27888228313399066
  (3, 101)	0.27888228313399066
  (3, 128)	0.19863672392259624
  :	:
  (95, 141)	0.42484474614924805
  (95, 112)	0.42484474614924805
  (95, 45)	0.42484474614924805
  (95, 74)	0.38970384828260646
  (95, 223)	0.3162701372700484
  (95, 134)	0.27709743719530117
  (95, 97)	0.20594459776752813
  (95, 33)	0.14762176669

In [6]:
def response(user_input):
    user_input_processed = preprocess_text(user_input)
    v_input = tfidf_vector.transform([user_input_processed])
    most_similar = cosine_similarity(v_input, v_corpus)
    most_similar_index = most_similar.argmax()
    
    return data['Answers'].iloc[most_similar_index]

In [17]:
response('what is mental illness')

'We all have mental health which is made up of our beliefs, thoughts, feelings and behaviours.'

In [7]:
import random

chatbot_greeting = [
    "Hello there, welcome to Orpheus Bot. pls ejoy your usage",
    "Hi user, This bot is created by oprheus, enjoy your usage",
    "Hi hi, How you dey my nigga",
    "Alaye mi, Abeg enjoy your usage",
    "Hey Hey, pls enjoy your usage"    
]

user_greeting = ["hi", "hello there", "hey", "hi there"]
exit_word = ['bye', 'thanks bye', 'exit', 'goodbye']


print(f'\t\t\t\t\tWelcome To Orpheus ChatBot\n\n')
while True:
    user_q = input('Pls ask your mental illness related question: ')
    if user_q in user_greeting:
        print(random.choice(chatbot_greeting))
    elif user_q in exit_word:
        print('Thank you for your usage. Bye')
        break
    else:
        responses = response(user_q)
        print(f'ChatBot:  {responses}')

					Welcome To Orpheus ChatBot


Hi user, This bot is created by oprheus, enjoy your usage
ChatBot:  Some people may advise you on good evidence of what works with the best of intentions, but it's important to find out what works best for you.
Thank you for your usage. Bye
